In [1]:
from sklearn import datasets
import numpy as np

from typing import *
import copy

In [2]:
iris = datasets.load_iris()

#shape
X = iris.data
y = iris.target

#making the dataset into binary for binary classification
binary_filter = y < 2
X = X[binary_filter]
y = y[binary_filter]

mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

X = (X - mean) / std

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (100, 4)
Shape of y: (100,)


In [3]:
#splitting into test and train
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=55)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (80, 4)
Shape of X_test: (20, 4)
Shape of Y_train: (80,)
Shape of Y_test: (20,)


In [4]:
X_train = X_train.T 
X_test = X_test.T

Y_train = Y_train.reshape(1, -1)
Y_test = Y_test.reshape(1,-1)

In [5]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (4, 80)
Shape of X_test: (4, 20)
Shape of Y_train: (1, 80)
Shape of Y_test: (1, 20)


In [6]:
def layer_sizes(X: np.ndarray, Y: np.ndarray, hidden_layers: int = 5) -> Tuple[int, int, int]:
    n_x = X.shape[0]  #input layer size
    n_y = Y.shape[0]  #output layer size
    n_h = hidden_layers  #number of hidden layers
    return (n_x, n_h, n_y)

In [7]:
sizes = layer_sizes(X_train,Y_train)
sizes

(4, 5, 1)

In [8]:
n_h = sizes[1]

In [9]:
def initialize_parameters(n_x: int, n_h: int, n_y: int) -> Dict[str, np.ndarray]:
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [10]:
def forward_propagation(X: np.ndarray, parameters: Dict[str, np.ndarray]) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    
    Z2 = np.clip(Z2, -50, 50)  #to avoid overflow
    
    A2 = 1 / (1 + np.exp(-Z2))
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache


In [11]:
def compute_cost(A2: np.ndarray, Y: np.ndarray) -> float:
    #A2: sigmoid output of the second activation -> (1, number of examples)
    m = Y.shape[1]
    A2 = np.clip(A2, 1e-10, 1 - 1e-10)  # avoiding log(0)
    logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), 1 - Y)
    cost = - np.sum(logprobs) / m
    
    cost = float(np.squeeze(cost))
    
    return cost

\[
\begin{align*}
dz^{[2]} &= a^{[2]} - y \\
dW^{[2]} &= dz^{[2]} a^{[1]T} \\
db^{[2]} &= dz^{[2]} \\
dz^{[1]} &= W^{[2]T} dz^{[2]} * g^{[1]'}(z^{[1]}) \\
dW^{[1]} &= dz^{[1]} x^T \\
db^{[1]} &= dz^{[1]}
\end{align*}
\]

\[
\begin{align*}
dZ^{[2]} &= A^{[2]} - Y \\
dW^{[2]} &= \frac{1}{m} dZ^{[2]} A^{[1]T} \\
db^{[2]} &= \frac{1}{m} \text{np.sum}(dZ^{[2]}, \text{axis}=1, \text{keepdims}=\text{True}) \\
dZ^{[1]} &= W^{[2]T} dZ^{[2]} * g^{[1]'}(Z^{[1]}) \\
dW^{[1]} &= \frac{1}{m} dZ^{[1]} X^T \\
db^{[1]} &= \frac{1}{m} \text{np.sum}(dZ^{[1]}, \text{axis}=1, \text{keepdims}=\text{True})
\end{align*}
\]


In [12]:
def backward_propagation(parameters: Dict[str, np.ndarray], cache: Dict[str, np.ndarray], X: np.ndarray, Y: np.ndarray) -> Dict[str, np.ndarray]:
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    A1 = cache['A1']
    A2 = cache['A2']
    
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))  #tanh derivative
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [13]:
def update_parameters(parameters: Dict[str, np.ndarray], grads: Dict[str, np.ndarray], learning_rate: float = 0.00001) -> Dict[str, np.ndarray]:
    W1 = copy.deepcopy(parameters['W1'])
    b1 = copy.deepcopy(parameters['b1'])
    W2 = copy.deepcopy(parameters['W2'])
    b2 = copy.deepcopy(parameters['b2'])
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


In [14]:
def nn_model(X: np.ndarray, Y: np.ndarray, n_h: int, num_iterations: int = 10000, print_cost: bool = True) -> Dict[str, np.ndarray]:
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)  #forward
        cost = compute_cost(A2, Y)  #cost
        grads = backward_propagation(parameters, cache, X, Y)  #backward
        
        parameters = update_parameters(parameters, grads, learning_rate=1.2)
    
        if print_cost and i % 1000 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return parameters


In [15]:
params = nn_model(X_train, Y_train, n_h)

Cost after iteration 0: 0.693151
Cost after iteration 1000: 0.000181
Cost after iteration 2000: 0.000089
Cost after iteration 3000: 0.000059
Cost after iteration 4000: 0.000044
Cost after iteration 5000: 0.000035
Cost after iteration 6000: 0.000029
Cost after iteration 7000: 0.000025
Cost after iteration 8000: 0.000022
Cost after iteration 9000: 0.000019


In [16]:
def predict(parameters: Dict[str, np.ndarray], X: np.ndarray) -> np.ndarray:
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5).astype(int)
    return predictions

In [17]:
def evaluate_model(parameters: Dict[str, np.ndarray], X_test: np.ndarray, Y_test: np.ndarray) -> float:
    predictions = predict(parameters, X_test)
    print("Predictions:", predictions)
    print("True labels:", Y_test)
    
    true_positive = np.sum((predictions == 1) & (Y_test == 1))
    true_negative = np.sum((predictions == 0) & (Y_test == 0))
    total = Y_test.size
    
    accuracy = (true_positive + true_negative) / total * 100
    
    return accuracy

In [18]:
evaluate_model(params, X_test, Y_test)

Predictions: [[0 1 1 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1]]
True labels: [[0 1 1 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1]]


100.0